# Collect results (Machine Translation)

In [ ]:
import numpy as np
import pandas as pd
from collections import defaultdict
from utils import detrend_ue
from pathlib import Path
import pathlib


methods_dict = {
    'MaximumSequenceProbability': 'MSP',
    'Perplexity': 'PPL',
    'MeanTokenEntropy': 'MTE',
    'MonteCarloSequenceEntropy': 'MCSE',
    'MonteCarloNormalizedSequenceEntropy': 'MCNSE',
    'LexicalSimilarity_rougeL': 'LSRL',
    'TokenSAR':'TokenSAR'
}

MODELS = {
    'llama': 'Llama 3.1 8B',
    'gemma': 'Gemma 2 9B',
    'eurollm': 'EuroLLM 9B',
}

DATASETS = {
    'metricx-metricx-24-hybrid-xxl-v2p6': [
        'wmt14_csen', 'wmt14_deen', 'wmt14_ruen', 'wmt14_fren',
        'wmt19_deen', 'wmt19_fien', 'wmt19_lten', 'wmt19_ruen',
    ],
    'XComet-XCOMET-XXL': [
        'wmt14_csen', 'wmt14_deen', 'wmt14_ruen', 'wmt14_fren',
        'wmt19_deen', 'wmt19_fien', 'wmt19_lten', 'wmt19_ruen',
    ],
    'Comet-wmt22-comet-da': [
        'wmt14_csen', 'wmt14_deen', 'wmt14_ruen', 'wmt14_fren',
        'wmt19_deen', 'wmt19_fien', 'wmt19_lten', 'wmt19_ruen',
    ]
}

METRICS = {
    'metricx-metricx-24-hybrid-xxl-v2p6': 'MetricX XXL',
    'XComet-XCOMET-XXL': 'XComet XXL',
    'Comet-wmt22-comet-da': 'Comet WMT22',
}


def main():
    rows = []  # will accumulate dicts with columns: model, dataset, metric, method, prr_score

    for metric_key, metric_name in METRICS.items():
        datasets = DATASETS[metric_key]
        

        for model_key, model_name in MODELS.items():
            ue_methods = list(methods_dict.values())

            ue_scores, _, _ = detrend_ue(
                datasets,
                model_key,
                [metric_key],
                ue_methods,
                methods_dict
            )

            for method_full, method_short in methods_dict.items():
                raw_scores = ue_scores[f"{method_short}_raw"]
                detr_scores = ue_scores[f"{method_short}_detr"]

                for dataset_name, raw, detr in zip(datasets, raw_scores, detr_scores):
                    # raw row (method name unchanged)
                    rows.append({
                        "model": model_name,
                        "dataset": dataset_name,
                        "metric": metric_name,
                        "method": method_short,
                        "prr_score": float(raw),
                    })
                    # detrended row (method-LINE)
                    rows.append({
                        "model": model_name,
                        "dataset": dataset_name,
                        "metric": metric_name,
                        "method": f"{method_short}-LINE",
                        "prr_score": float(detr),
                    })

    # write single CSV with all metrics/models/datasets/methods
    df = pd.DataFrame(rows, columns=["model", "dataset", "metric", "method", "prr_score"])
    out_path = Path("results") / "mt_results.csv"
    df.to_csv(out_path, index=False)
    print(f"Wrote {out_path.resolve()} with {len(df)} rows.")

if __name__ == "__main__":
    main()


In [ ]:
import pandas as pd


df = pd.read_csv("results/mt_results.csv")


import numpy as np
from IPython.display import display, Markdown


methods_dict = {
    'MaximumSequenceProbability': 'MSP',
    'Perplexity': 'PPL',
    'MeanTokenEntropy': 'MTE',
    'MonteCarloSequenceEntropy': 'MCSE',
    'MonteCarloNormalizedSequenceEntropy': 'MCNSE',
    'LexicalSimilarity_rougeL': 'LSRL',
    'TokenSAR':'TokenSAR'
}
method_order = [v for v in methods_dict.values()]
method_display_order = [m for pair in zip(method_order, [m + "-LINE" for m in method_order]) for m in pair]

MODELS = {
    'llama': 'Llama 3.1 8B',
    'gemma': 'Gemma 2 9B',
    'eurollm': 'EuroLLM 9B',
}
model_display_order = list(MODELS.values())

DATASETS = {
    'metricx-metricx-24-hybrid-xxl-v2p6': [
        'wmt14_csen','wmt14_deen','wmt14_ruen','wmt14_fren',
        'wmt19_deen','wmt19_fien','wmt19_lten','wmt19_ruen',
    ],
    'XComet-XCOMET-XXL': [
        'wmt14_csen','wmt14_deen','wmt14_ruen','wmt14_fren',
        'wmt19_deen','wmt19_fien','wmt19_lten','wmt19_ruen',
    ],
    'Comet-wmt22-comet-da': [
        'wmt14_csen','wmt14_deen','wmt14_ruen','wmt14_fren',
        'wmt19_deen','wmt19_fien','wmt19_lten','wmt19_ruen',
    ]
}

# ---- highlighter: best = bold, second = underline (ties handled) ----
def highlight_best_second(col: pd.Series):
    if col.dtype.kind not in "fi":
        return [''] * len(col)
    vals = col.astype(float)
    uniq_sorted = np.unique(vals[~vals.isna()])[::-1]  # desc unique
    best = uniq_sorted[0] if len(uniq_sorted) > 0 else np.nan
    second = uniq_sorted[1] if len(uniq_sorted) > 1 else np.nan

    styles = []
    for v in vals:
        if pd.isna(v):
            styles.append('')
        elif np.isclose(v, best, rtol=1e-9, atol=1e-12):
            styles.append('font-weight: bold;')
        elif not np.isnan(second) and np.isclose(v, second, rtol=1e-9, atol=1e-12):
            styles.append('text-decoration: underline;')
        else:
            styles.append('')
    return styles

def show_metric_tables(df_in: pd.DataFrame, metric_name: str):
    display(Markdown(f"## {metric_name}"))
    metric_df = df_in[df_in["metric"] == metric_name].copy()

    candidate_order = None
    for ds_list in DATASETS.values():
        if set(ds_list).issubset(set(metric_df["dataset"].unique())):
            candidate_order = ds_list
            break
    if candidate_order is None:
        candidate_order = list(metric_df["dataset"].unique())

    for model_name in model_display_order:
        block = metric_df[metric_df["model"] == model_name]
        if block.empty:
            continue

        piv = block.pivot_table(
            index="method", columns="dataset", values="prr_score", aggfunc="mean"
        )
        piv = piv.reindex(index=method_display_order)
        piv = piv.reindex(columns=[c for c in candidate_order if c in piv.columns])

        styled = (
            piv.round(3)
               .style
               .apply(highlight_best_second, axis=0)
               .set_caption(model_name)
        )
        display(styled)

for metric_name in df["metric"].unique():
    show_metric_tables(df, metric_name)


## MetricX XXL

dataset,wmt14_csen,wmt14_deen,wmt14_ruen,wmt14_fren,wmt19_deen,wmt19_fien,wmt19_lten,wmt19_ruen
method,,,,,,,,
MSP,0.215000,0.225000,0.313000,0.194000,0.225000,0.084000,0.119000,0.264000
MSP-LINE,0.396000,0.422000,0.436000,0.310000,0.403000,0.412000,0.382000,0.355000
PPL,0.429000,0.450000,0.414000,0.319000,0.390000,0.482000,0.434000,0.311000
PPL-LINE,0.484000,0.473000,0.485000,0.357000,0.404000,0.482000,0.436000,0.396000
MTE,0.475000,0.477000,0.459000,0.393000,0.427000,0.520000,0.487000,0.356000
MTE-LINE,0.537000,0.514000,0.538000,0.435000,0.471000,0.509000,0.489000,0.454000
MCSE,0.157000,0.140000,0.202000,0.163000,0.143000,-0.002000,0.065000,0.213000
MCSE-LINE,0.292000,0.291000,0.304000,0.246000,0.318000,0.295000,0.288000,0.275000
MCNSE,0.423000,0.378000,0.404000,0.324000,0.383000,0.393000,0.438000,0.324000


dataset,wmt14_csen,wmt14_deen,wmt14_ruen,wmt14_fren,wmt19_deen,wmt19_fien,wmt19_lten,wmt19_ruen
method,,,,,,,,
MSP,0.189000,0.219000,0.288000,0.133000,0.279000,0.058000,0.241000,0.265000
MSP-LINE,0.392000,0.446000,0.412000,0.290000,0.446000,0.352000,0.349000,0.386000
PPL,0.425000,0.471000,0.405000,0.326000,0.419000,0.408000,0.331000,0.340000
PPL-LINE,0.429000,0.479000,0.427000,0.330000,0.427000,0.407000,0.342000,0.371000
MTE,0.450000,0.475000,0.419000,0.359000,0.440000,0.454000,0.335000,0.350000
MTE-LINE,0.463000,0.495000,0.460000,0.371000,0.467000,0.446000,0.373000,0.414000
MCSE,0.110000,0.155000,0.213000,0.120000,0.201000,-0.027000,0.172000,0.229000
MCSE-LINE,0.308000,0.359000,0.328000,0.276000,0.401000,0.266000,0.245000,0.352000
MCNSE,0.383000,0.431000,0.402000,0.330000,0.429000,0.363000,0.319000,0.381000


dataset,wmt14_csen,wmt14_deen,wmt14_ruen,wmt14_fren,wmt19_deen,wmt19_fien,wmt19_lten,wmt19_ruen
method,,,,,,,,
MSP,0.127000,0.226000,0.297000,0.111000,0.230000,0.041000,0.155000,0.284000
MSP-LINE,0.324000,0.444000,0.445000,0.265000,0.428000,0.339000,0.312000,0.379000
PPL,0.512000,0.521000,0.454000,0.415000,0.481000,0.445000,0.403000,0.334000
PPL-LINE,0.515000,0.529000,0.458000,0.419000,0.483000,0.436000,0.423000,0.371000
MTE,0.541000,0.537000,0.477000,0.456000,0.498000,0.486000,0.424000,0.361000
MTE-LINE,0.548000,0.547000,0.470000,0.465000,0.512000,0.470000,0.467000,0.420000
MCSE,0.200000,0.237000,0.283000,0.164000,0.247000,0.098000,0.254000,0.273000
MCSE-LINE,0.417000,0.421000,0.389000,0.360000,0.402000,0.357000,0.356000,0.343000
MCNSE,0.230000,0.330000,0.295000,0.223000,0.276000,0.324000,0.264000,0.214000


## XComet XXL

...

dataset,wmt14_csen,wmt14_deen,wmt14_ruen,wmt14_fren,wmt19_deen,wmt19_fien,wmt19_lten,wmt19_ruen
method,,,,,,,,
MSP,0.133000,0.308000,0.379000,0.205000,0.268000,-0.027000,0.058000,0.355000
MSP-LINE,0.237000,0.384000,0.425000,0.248000,0.383000,0.261000,0.189000,0.365000
PPL,0.394000,0.405000,0.375000,0.328000,0.425000,0.397000,0.331000,0.314000
PPL-LINE,0.412000,0.435000,0.499000,0.373000,0.437000,0.384000,0.336000,0.390000
MTE,0.434000,0.417000,0.389000,0.345000,0.430000,0.448000,0.386000,0.322000
MTE-LINE,0.462000,0.462000,0.511000,0.417000,0.473000,0.421000,0.381000,0.440000
MCSE,0.185000,0.305000,0.348000,0.263000,0.303000,-0.009000,0.080000,0.337000
MCSE-LINE,0.307000,0.365000,0.383000,0.331000,0.384000,0.279000,0.200000,0.322000
MCNSE,0.191000,0.277000,0.229000,0.171000,0.259000,0.300000,0.225000,0.212000


## Comet WMT22

dataset,wmt14_csen,wmt14_deen,wmt14_ruen,wmt14_fren,wmt19_deen,wmt19_fien,wmt19_lten,wmt19_ruen
method,,,,,,,,
MSP,0.424000,0.394000,0.453000,0.349000,0.458000,0.186000,0.288000,0.431000
MSP-LINE,0.469000,0.487000,0.481000,0.398000,0.511000,0.467000,0.473000,0.414000
PPL,0.417000,0.455000,0.371000,0.314000,0.407000,0.516000,0.475000,0.315000
PPL-LINE,0.521000,0.515000,0.526000,0.412000,0.459000,0.521000,0.495000,0.464000
MTE,0.445000,0.479000,0.406000,0.374000,0.418000,0.540000,0.518000,0.331000
MTE-LINE,0.576000,0.563000,0.589000,0.484000,0.546000,0.561000,0.555000,0.526000
MCSE,0.365000,0.322000,0.347000,0.296000,0.361000,0.085000,0.201000,0.359000
MCSE-LINE,0.376000,0.356000,0.328000,0.280000,0.385000,0.318000,0.355000,0.320000
MCNSE,0.481000,0.435000,0.401000,0.358000,0.432000,0.455000,0.483000,0.353000


dataset,wmt14_csen,wmt14_deen,wmt14_ruen,wmt14_fren,wmt19_deen,wmt19_fien,wmt19_lten,wmt19_ruen
method,,,,,,,,
MSP,0.398000,0.373000,0.425000,0.286000,0.490000,0.185000,0.350000,0.401000
MSP-LINE,0.482000,0.501000,0.473000,0.383000,0.533000,0.422000,0.360000,0.412000
PPL,0.436000,0.484000,0.382000,0.361000,0.445000,0.456000,0.304000,0.305000
PPL-LINE,0.457000,0.506000,0.496000,0.400000,0.466000,0.456000,0.321000,0.355000
MTE,0.443000,0.489000,0.384000,0.371000,0.436000,0.487000,0.302000,0.304000
MTE-LINE,0.492000,0.537000,0.529000,0.436000,0.506000,0.493000,0.360000,0.401000
MCSE,0.319000,0.308000,0.353000,0.283000,0.406000,0.092000,0.287000,0.361000
MCSE-LINE,0.394000,0.431000,0.375000,0.352000,0.472000,0.307000,0.294000,0.387000
MCNSE,0.442000,0.502000,0.417000,0.370000,0.473000,0.413000,0.352000,0.373000


dataset,wmt14_csen,wmt14_deen,wmt14_ruen,wmt14_fren,wmt19_deen,wmt19_fien,wmt19_lten,wmt19_ruen
method,,,,,,,,
MSP,0.293000,0.332000,0.423000,0.238000,0.401000,0.162000,0.277000,0.430000
MSP-LINE,0.374000,0.461000,0.500000,0.342000,0.507000,0.394000,0.341000,0.436000
PPL,0.505000,0.505000,0.434000,0.440000,0.516000,0.484000,0.357000,0.325000
PPL-LINE,0.531000,0.531000,0.537000,0.473000,0.536000,0.486000,0.403000,0.394000
MTE,0.525000,0.517000,0.456000,0.466000,0.511000,0.512000,0.372000,0.341000
MTE-LINE,0.572000,0.553000,0.564000,0.520000,0.575000,0.521000,0.450000,0.453000
MCSE,0.351000,0.359000,0.424000,0.283000,0.414000,0.211000,0.343000,0.422000
MCSE-LINE,0.462000,0.466000,0.456000,0.396000,0.487000,0.400000,0.375000,0.389000
MCNSE,0.225000,0.357000,0.282000,0.221000,0.338000,0.355000,0.277000,0.245000


# Write to latex

In [ ]:
import os
import numpy as np
import pandas as pd

methods_dict = {
    'MaximumSequenceProbability': 'MSP',
    'Perplexity': 'PPL',
    'MeanTokenEntropy': 'MTE',
    'MonteCarloSequenceEntropy': 'MCSE',
    'MonteCarloNormalizedSequenceEntropy': 'MCNSE',
    'LexicalSimilarity_rougeL': 'LSRL',
    'TokenSAR':'TokenSAR'
}
method_order = [v for v in methods_dict.values()]
method_display_order = [m for pair in zip(method_order, [m + "-LINE" for m in method_order]) for m in pair]

MODELS = {
    'llama': 'Llama 3.1 8B',
    'gemma': 'Gemma 2 9B',
    'eurollm': 'EuroLLM 9B',
}
model_display_order = list(MODELS.values())

DATASETS = {
    'metricx-metricx-24-hybrid-xxl-v2p6': [
        'wmt14_csen','wmt14_deen','wmt14_ruen','wmt14_fren',
        'wmt19_deen','wmt19_fien','wmt19_lten','wmt19_ruen',
    ],
    'XComet-XCOMET-XXL': [
        'wmt14_csen','wmt14_deen','wmt14_ruen','wmt14_fren',
        'wmt19_deen','wmt19_fien','wmt19_lten','wmt19_ruen',
    ],
    'Comet-wmt22-comet-da': [
        'wmt14_csen','wmt14_deen','wmt14_ruen','wmt14_fren',
        'wmt19_deen','wmt19_fien','wmt19_lten','wmt19_ruen',
    ]
}

def _choose_dataset_order(metric_df: pd.DataFrame) -> list[str]:
    cand = None
    for ds_list in DATASETS.values():
        if set(ds_list).issubset(set(metric_df["dataset"].unique())):
            cand = ds_list
            break
    if cand is None:
        cand = list(metric_df["dataset"].unique())
    return cand

def _format_with_rankings(col: pd.Series, ndigits: int = 3) -> pd.Series:
    vals = pd.to_numeric(col, errors="coerce")
    uniq = np.unique(vals[~vals.isna()])
    if uniq.size == 0:
        best = second = np.nan
    else:
        uniq.sort()           # ascending
        uniq = uniq[::-1]     # descending
        best = uniq[0]
        second = uniq[1] if uniq.size > 1 else np.nan

    out = []
    for v in vals:
        if pd.isna(v):
            out.append("--")
            continue
        s = f"{v:.{ndigits}f}"
        if np.isfinite(best) and np.isclose(v, best, rtol=1e-9, atol=1e-12):
            out.append(r"\textbf{" + s + "}")
        elif np.isfinite(second) and np.isclose(v, second, rtol=1e-9, atol=1e-12):
            out.append(r"\underline{" + s + "}")
        else:
            out.append(s)
    return pd.Series(out, index=col.index)

def generate_latex_tables(
    df_in: pd.DataFrame,
    value_col: str = "prr_score",
    out_dir: str = "latex_tables",
    ndigits: int = 2,
    table_env: str = "table",     
    use_booktabs: bool = True
) -> dict:
    """
    Generate LaTeX tables for each (metric, model) slice.
    Returns a dict: {(metric, model): filepath}.
    """
    os.makedirs(out_dir, exist_ok=True)
    paths = {}

    for metric_name in df_in["metric"].unique():
        metric_df = df_in[df_in["metric"] == metric_name].copy()
        dataset_order = _choose_dataset_order(metric_df)

        for model_name in model_display_order:
            block = metric_df[metric_df["model"] == model_name]
            if block.empty:
                continue

            piv = block.pivot_table(
                index="method", columns="dataset", values=value_col, aggfunc="mean"
            )

            # enforce display order
            piv = piv.reindex(index=method_display_order)
            piv = piv.reindex(columns=[c for c in dataset_order if c in piv.columns])

            fmt = piv.apply(_format_with_rankings, axis=0, ndigits=ndigits)

            fmt = fmt.fillna("--")

            colfmt = "l" + "r" * fmt.shape[1]

            caption = f"{model_name} — {metric_name}"
            label = f"tab:{metric_name.replace(' ', '_')}__{model_name.replace(' ', '_')}"

            latex = fmt.to_latex(
                escape=False,           
                index=True,
                caption=caption,
                label=label,
                na_rep="--",
                column_format=colfmt,
                bold_rows=False,
                longtable=False
            )

            if use_booktabs:
                pass

            latex = latex.replace("\\begin{tabular}", "\\begin{tabular}")
            if "method" in (piv.index.names or []) or True:
                lines = latex.splitlines()
                try:
                    top_idx = next(i for i,l in enumerate(lines) if r"\toprule" in l)
                    header_idx = top_idx + 1
                    header_line = lines[header_idx]
                    if header_line.startswith("{}"):
                        lines[header_idx] = header_line.replace("{}", r"\textbf{Method}", 1)
                    elif header_line.startswith("&"):
                        lines[header_idx] = r"\textbf{Method} " + header_line
                    latex = "\n".join(lines)
                except StopIteration:
                    pass

            # write out
            safe_metric = "".join(ch if ch.isalnum() or ch in "-_." else "_" for ch in metric_name)
            safe_model = "".join(ch if ch.isalnum() or ch in "-_." else "_" for ch in model_name)
            fname = f"{safe_metric}__{safe_model}.tex"
            fpath = os.path.join(out_dir, fname)
            with open(fpath, "w", encoding="utf-8") as f:
                f.write(latex)

            paths[(metric_name, model_name)] = fpath

    return paths


df= pd.read_csv('results/mt_results.csv')
paths = generate_latex_tables(df)
for (metric, model), p in paths.items():
    print(f"{metric} | {model} -> {p}")


MetricX XXL | Llama 3.1 8B -> latex_tables/MetricX_XXL__Llama_3.1_8B.tex
MetricX XXL | Gemma 2 9B -> latex_tables/MetricX_XXL__Gemma_2_9B.tex
MetricX XXL | EuroLLM 9B -> latex_tables/MetricX_XXL__EuroLLM_9B.tex
XComet XXL | Llama 3.1 8B -> latex_tables/XComet_XXL__Llama_3.1_8B.tex
XComet XXL | Gemma 2 9B -> latex_tables/XComet_XXL__Gemma_2_9B.tex
XComet XXL | EuroLLM 9B -> latex_tables/XComet_XXL__EuroLLM_9B.tex
Comet WMT22 | Llama 3.1 8B -> latex_tables/Comet_WMT22__Llama_3.1_8B.tex
Comet WMT22 | Gemma 2 9B -> latex_tables/Comet_WMT22__Gemma_2_9B.tex
Comet WMT22 | EuroLLM 9B -> latex_tables/Comet_WMT22__EuroLLM_9B.tex


# Collect results (Summarization and Math Reasoning)

In [ ]:
from utils import detrend_ue_w_quality
from pathlib import Path


MODELS = {
    'llama': 'Llama 3.1 8B',
    'gemma': 'Gemma 2 9B',
}

DATASETS = {
    'Accuracy': [
        'gsm8k'
    ],
    'AlignScoreInputOutput': [
        'xsum'
    ]
}

METRICS = {
    'Accuracy': 'Accuracy',
    'AlignScoreInputOutput': 'Align Score',
}


def main():
    rows = []  

    for metric_key, metric_name in METRICS.items():
        datasets = DATASETS[metric_key]
        ue_methods = list(methods_dict.values())

        for model_key, model_name in MODELS.items():
            ue_scores, _, _ = detrend_ue_w_quality(
                datasets,
                model_key,
                [metric_key],
                ue_methods,
                methods_dict
            )

            for method_full, method_short in methods_dict.items():
                raw_scores = ue_scores[f"{method_short}_raw"]
                detr_scores = ue_scores[f"{method_short}_detr"]

                for dataset_name, raw, detr in zip(datasets, raw_scores, detr_scores):
                    rows.append({
                        "model": model_name,
                        "dataset": dataset_name,
                        "metric": metric_name,
                        "method": method_short,
                        "prr_score": float(raw),
                    })
                    # detrended row (method-LINE)
                    rows.append({
                        "model": model_name,
                        "dataset": dataset_name,
                        "metric": metric_name,
                        "method": f"{method_short}-LINE",
                        "prr_score": float(detr),
                    })

    df = pd.DataFrame(rows, columns=["model", "dataset", "metric", "method", "prr_score"])
    out_path = Path("results") / "sum_mr_results.csv"
    df.to_csv(out_path, index=False)
    print(f"Wrote {out_path.resolve()} with {len(df)} rows.")

if __name__ == "__main__":
    main()


In [ ]:


df = pd.read_csv("results/sum_mr_results.csv")


import numpy as np
from IPython.display import display, Markdown


methods_dict = {
    'MaximumSequenceProbability': 'MSP',
    'Perplexity': 'PPL',
    'MeanTokenEntropy': 'MTE',
    'MonteCarloSequenceEntropy': 'MCSE',
    'MonteCarloNormalizedSequenceEntropy': 'MCNSE',
    'LexicalSimilarity_rougeL': 'LSRL',
    'TokenSAR':'TokenSAR'
}
method_order = [v for v in methods_dict.values()]
method_display_order = [m for pair in zip(method_order, [m + "-LINE" for m in method_order]) for m in pair]

MODELS = {
    'llama': 'Llama 3.1 8B',
    'gemma': 'Gemma 2 9B',
}
model_display_order = list(MODELS.values())


DATASETS = {
    'Accuracy': [
        'gsm8k'
    ],
    'AlignScoreInputOutput': [
        'xsum'
    ]
}

METRICS = {
    'Accuracy': 'Accuracy',
    'AlignScoreInputOutput': 'Align Score',
}

def highlight_best_second(col: pd.Series):
    if col.dtype.kind not in "fi":
        return [''] * len(col)
    vals = col.astype(float)
    uniq_sorted = np.unique(vals[~vals.isna()])[::-1]  # desc unique
    best = uniq_sorted[0] if len(uniq_sorted) > 0 else np.nan
    second = uniq_sorted[1] if len(uniq_sorted) > 1 else np.nan

    styles = []
    for v in vals:
        if pd.isna(v):
            styles.append('')
        elif np.isclose(v, best, rtol=1e-9, atol=1e-12):
            styles.append('font-weight: bold;')
        elif not np.isnan(second) and np.isclose(v, second, rtol=1e-9, atol=1e-12):
            styles.append('text-decoration: underline;')
        else:
            styles.append('')
    return styles

def show_metric_tables(df_in: pd.DataFrame, metric_name: str):
    display(Markdown(f"## {metric_name}"))
    metric_df = df_in[df_in["metric"] == metric_name].copy()

    candidate_order = None
    for ds_list in DATASETS.values():
        if set(ds_list).issubset(set(metric_df["dataset"].unique())):
            candidate_order = ds_list
            break
    if candidate_order is None:
        candidate_order = list(metric_df["dataset"].unique())

    for model_name in model_display_order:
        block = metric_df[metric_df["model"] == model_name]
        if block.empty:
            continue

        piv = block.pivot_table(
            index="method", columns="dataset", values="prr_score", aggfunc="mean"
        )
        piv = piv.reindex(index=method_display_order)
        piv = piv.reindex(columns=[c for c in candidate_order if c in piv.columns])

        styled = (
            piv.round(3)
               .style
               .apply(highlight_best_second, axis=0)
               .set_caption(model_name)
        )
        display(styled)

for metric_name in df["metric"].unique():
    show_metric_tables(df, metric_name)


## Accuracy

dataset,gsm8k
method,
MSP,0.324000
MSP-LINE,0.329000
PPL,0.303000
PPL-LINE,0.377000
MTE,0.339000
MTE-LINE,0.400000
MCSE,0.351000
MCSE-LINE,0.352000
MCNSE,0.343000


dataset,gsm8k
method,
MSP,0.303000
MSP-LINE,0.296000
PPL,0.248000
PPL-LINE,0.358000
MTE,0.292000
MTE-LINE,0.399000
MCSE,0.393000
MCSE-LINE,0.399000
MCNSE,0.356000


## Align Score

dataset,xsum
method,
MSP,0.328000
MSP-LINE,0.357000
PPL,0.369000
PPL-LINE,0.366000
MTE,0.357000
MTE-LINE,0.350000
MCSE,0.033000
MCSE-LINE,0.043000
MCNSE,0.024000


dataset,xsum
method,
MSP,0.351000
MSP-LINE,0.378000
PPL,0.354000
PPL-LINE,0.373000
MTE,0.333000
MTE-LINE,0.356000
MCSE,0.003000
MCSE-LINE,0.032000
MCNSE,0.016000


In [ ]:
import os
import numpy as np
import pandas as pd
df = pd.read_csv("results/sum_mr_results.csv")

import os
import numpy as np
import pandas as pd

methods_dict = {
    'MaximumSequenceProbability': 'MSP',
    'Perplexity': 'PPL',
    'MeanTokenEntropy': 'MTE',
    'MonteCarloSequenceEntropy': 'MCSE',
    'MonteCarloNormalizedSequenceEntropy': 'MCNSE',
    'LexicalSimilarity_rougeL': 'LSRL',
    'TokenSAR':'TokenSAR'
}
method_order = [v for v in methods_dict.values()]
method_display_order = [m for pair in zip(method_order, [m + "-LINE" for m in method_order]) for m in pair]

MODELS = {
    'llama': 'Llama 3.1 8B',
    'gemma': 'Gemma 2 9B',
}
model_display_order = list(MODELS.values())

COLUMN_SPECS = [
        {"col": "xsum",  "metric": "Align Score", "pretty": "Align Score"},

    {"col": "gsm8k", "metric": "Accuracy", "pretty": "Accuracy"},
]

def _format_with_rankings(col: pd.Series, ndigits: int = 3) -> pd.Series:
    """Return a string-formatted column with best bolded and second underlined (ties respected)."""
    vals = pd.to_numeric(col, errors="coerce")
    uniq = np.unique(vals[~vals.isna()])
    if uniq.size == 0:
        best = second = np.nan
    else:
        uniq.sort()
        uniq = uniq[::-1]  # desc
        best = uniq[0]
        second = uniq[1] if uniq.size > 1 else np.nan

    out = []
    for v in vals:
        if pd.isna(v):
            out.append("--")
            continue
        s = f"{v:.{ndigits}f}"
        if np.isfinite(best) and np.isclose(v, best, rtol=1e-9, atol=1e-12):
            out.append(r"\textbf{" + s + "}")
        elif np.isfinite(second) and np.isclose(v, second, rtol=1e-9, atol=1e-12):
            out.append(r"\underline{" + s + "}")
        else:
            out.append(s)
    return pd.Series(out, index=col.index)

def generate_latex_tables_mixed_metrics(
    df_in: pd.DataFrame,
    value_col: str = "prr_score",
    out_dir: str = "latex_tables",
    ndigits: int = 2,
) -> dict:
  
    os.makedirs(out_dir, exist_ok=True)
    paths = {}

    for model_name in model_display_order:
        cols = {}
        for spec in COLUMN_SPECS:
            sub = df_in[
                (df_in["model"] == model_name)
                & (df_in["dataset"] == spec["col"])
                & (df_in["metric"] == spec["metric"])
            ]
            s = (
                sub.pivot_table(
                    index="method", values=value_col, aggfunc="mean"
                )[value_col]
                if not sub.empty else pd.Series(dtype=float)
            )
            cols[spec["col"]] = s

        block = pd.DataFrame(cols)

        block = block.reindex(index=method_display_order)
        block = block.reindex(columns=[spec["col"] for spec in COLUMN_SPECS])

        fmt = block.apply(_format_with_rankings, axis=0, ndigits=ndigits).fillna("--")

        colfmt = "l" + "r" * fmt.shape[1]

        col_metric_parts = [f"{spec['col']} ({spec['pretty']})" for spec in COLUMN_SPECS]
        caption = f"{model_name} — " + " vs. ".join(col_metric_parts)
        label = f"tab:{model_name.replace(' ', '_')}__gsm8k_xsum"

        latex = fmt.to_latex(
            escape=False,          
            index=True,
            caption=caption,
            label=label,
            na_rep="--",
            column_format=colfmt,
            bold_rows=False,
            longtable=False
        )

        lines = latex.splitlines()
        try:
            top_idx = next(i for i, l in enumerate(lines) if r"\toprule" in l)
            header_idx = top_idx + 1
            if lines[header_idx].startswith("{}"):
                lines[header_idx] = lines[header_idx].replace("{}", r"\textbf{Method}", 1)
            elif lines[header_idx].startswith("&"):
                lines[header_idx] = r"\textbf{Method} " + lines[header_idx]
            latex = "\n".join(lines)
        except StopIteration:
            pass

        safe_model = "".join(ch if ch.isalnum() or ch in "-_." else "_" for ch in model_name)
        fname = f"{safe_model}__gsm8k_xsum.tex"
        fpath = os.path.join(out_dir, fname)
        with open(fpath, "w", encoding="utf-8") as f:
            f.write(latex)

        paths[model_name] = fpath

    return paths

paths = generate_latex_tables_mixed_metrics(df)
for model, p in paths.items():
    print(model, "->", p)


Llama 3.1 8B -> latex_tables/Llama_3.1_8B__gsm8k_xsum.tex
Gemma 2 9B -> latex_tables/Gemma_2_9B__gsm8k_xsum.tex


# Results with smaller sample size

In [ ]:
from utils import detrend_ue_w_quality

import numpy as np
import pandas as pd
from collections import defaultdict
from pathlib import Path


methods_dict = {
    'MaximumSequenceProbability': 'MSP',
    'Perplexity': 'PPL',
    'MeanTokenEntropy': 'MTE',
    'MonteCarloSequenceEntropy': 'MCSE',
    'MonteCarloNormalizedSequenceEntropy': 'MCNSE',
    'LexicalSimilarity_rougeL': 'LSRL',
    'TokenSAR':'TokenSAR'
}

MODELS = {
    'llama': 'Llama 3.1 8B',
    'gemma': 'Gemma 2 9B',
}

DATASETS = {
    'Accuracy': [
        'gsm8k'
    ],
    'AlignScoreInputOutput': [
        'xsum'
    ]
}

METRICS = {
    'Accuracy': 'Accuracy',
    'AlignScoreInputOutput': 'Align Score',
}


def main():
    rows = []  
    for metric_key, metric_name in METRICS.items():
        datasets = DATASETS[metric_key]
        ue_methods = list(methods_dict.values())

        for model_key, model_name in MODELS.items():
            ue_scores, _, _ = detrend_ue_w_quality(
                datasets,
                model_key,
                [metric_key],
                ue_methods,
                methods_dict,
                quality_fit_sample_size=500
            )

            for method_full, method_short in methods_dict.items():
                raw_scores = ue_scores[f"{method_short}_raw"]
                detr_scores = ue_scores[f"{method_short}_detr"]

                for dataset_name, raw, detr in zip(datasets, raw_scores, detr_scores):
                    # raw row (method name unchanged)
                    rows.append({
                        "model": model_name,
                        "dataset": dataset_name,
                        "metric": metric_name,
                        "method": method_short,
                        "prr_score": float(raw),
                    })
                    # detrended row (method-LINE)
                    rows.append({
                        "model": model_name,
                        "dataset": dataset_name,
                        "metric": metric_name,
                        "method": f"{method_short}-LINE",
                        "prr_score": float(detr),
                    })

    # write single CSV with all metrics/models/datasets/methods
    df = pd.DataFrame(rows, columns=["model", "dataset", "metric", "method", "prr_score"])
    out_path = Path("results") / "sum_mr_results_500.csv"
    df.to_csv(out_path, index=False)
    print(f"Wrote {out_path.resolve()} with {len(df)} rows.")

if __name__ == "__main__":
    main()


In [ ]:
import pandas as pd


sample_500 = pd.read_csv("results/sum_mr_results_500.csv")

sample_full = pd.read_csv("results/sum_mr_results.csv")



import pandas as pd
import numpy as np
from IPython.display import display, Markdown

# expects these to already exist:
# sample_full, sample_500  with columns: model, dataset, metric, method, prr_score

# ---- split out the three variants ----
cols = ["model","dataset","metric","method","prr_score"]
full_raw  = sample_full.loc[~sample_full["method"].str.endswith("-LINE"), cols].copy()
full_detr = sample_full.loc[ sample_full["method"].str.endswith("-LINE"), cols].copy()
s500_detr = sample_500.loc[ sample_500["method"].str.endswith("-LINE"), cols].copy()

# normalize to base method name (no suffix)
for df in (full_detr, s500_detr):
    df["method_base"] = df["method"].str.replace("-LINE","", regex=False)
full_raw["method_base"] = full_raw["method"]

full_raw["variant"]  = "raw"
s500_detr["variant"] = "500"
full_detr["variant"] = "full"

full_raw["method_display"]  = full_raw["method_base"]
s500_detr["method_display"] = s500_detr["method_base"] + "-LINE (500 sample)"
full_detr["method_display"] = full_detr["method_base"] + "-LINE (Full sample)"

full_raw["variant_order"]  = 0
s500_detr["variant_order"] = 1
full_detr["variant_order"] = 2

combined = pd.concat([full_raw, s500_detr, full_detr], ignore_index=True)

method_order = ["MSP","PPL","MTE","MCSE","MCNSE","LSRL"]
present_methods = [m for m in method_order if m in combined["method_base"].unique()]


if not present_methods:
    present_methods = list(combined["method_base"].unique())

def highlight_best_second(col: pd.Series):
    if col.dtype.kind not in "fi":
        return [''] * len(col)
    vals = col.astype(float)
    uniq = np.unique(vals[~vals.isna()])[::-1]
    best = uniq[0] if len(uniq) else np.nan
    second = uniq[1] if len(uniq) > 1 else np.nan
    out = []
    for v in vals:
        if pd.isna(v): out.append('')
        elif np.isclose(v, best): out.append('font-weight: bold;')
        elif not np.isnan(second) and np.isclose(v, second): out.append('text-decoration: underline;')
        else: out.append('')
    return out

for model in combined["model"].unique():
    block = combined[combined["model"] == model].copy()

    dataset_order = sorted(block["dataset"].unique())

    block["method_base"] = pd.Categorical(block["method_base"], categories=present_methods, ordered=True)
    block = block.sort_values(["method_base","variant_order"])

    piv = (block
           .pivot_table(index=["method_base","variant_order","method_display"],
                        columns="dataset", values="prr_score", aggfunc="mean")
           .reindex(columns=dataset_order)
           .sort_index(level=[0,1])
          )

    piv = piv.reset_index().set_index("method_display").drop(columns=["method_base","variant_order"])
    styled = (piv.round(4)
                 .style
                 .apply(highlight_best_second, axis=0)
                 .set_caption(model))

    display(Markdown(f"### {model}"))
    display(styled)



/tmp/ipykernel_4024775/702947106.py:77: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  piv = (block


### Llama 3.1 8B

dataset,gsm8k,xsum
method_display,,
MSP,0.324400,0.327600
MSP-LINE (500 sample),0.331800,0.357400
MSP-LINE (Full sample),0.328900,0.356900
PPL,0.302600,0.369100
PPL-LINE (500 sample),0.377200,0.366100
PPL-LINE (Full sample),0.376500,0.365600
MTE,0.339300,0.356900
MTE-LINE (500 sample),0.391400,0.353700
MTE-LINE (Full sample),0.399800,0.350400


/tmp/ipykernel_4024775/702947106.py:77: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  piv = (block


### Gemma 2 9B

dataset,gsm8k,xsum
method_display,,
MSP,0.302800,0.350700
MSP-LINE (500 sample),0.294700,0.378200
MSP-LINE (Full sample),0.296400,0.377700
PPL,0.248300,0.354100
PPL-LINE (500 sample),0.357000,0.372000
PPL-LINE (Full sample),0.357600,0.372600
MTE,0.291500,0.333200
MTE-LINE (500 sample),0.399200,0.355600
MTE-LINE (Full sample),0.398600,0.356000


# Results with 1-3 degrees polynomial fits

In [ ]:
import numpy as np
import pandas as pd
from collections import defaultdict
from utils import detrend_ue_degreed 
from pathlib import Path

methods_dict = {
    'MaximumSequenceProbability': 'MSP',
    'Perplexity': 'PPL',
    'MeanTokenEntropy': 'MTE',
    'MonteCarloSequenceEntropy': 'MCSE',
    'MonteCarloNormalizedSequenceEntropy': 'MCNSE',
    'LexicalSimilarity_rougeL': 'LSRL',
}

MODELS = {
    'llama': 'Llama 3.1 8B',
    'gemma': 'Gemma 2 9B',
    'eurollm': 'EuroLLM 9B',
}

DATASETS = {
    'metricx-metricx-24-hybrid-xxl-v2p6': [
        'wmt14_csen', 'wmt14_deen', 'wmt14_ruen', 'wmt14_fren',
        'wmt19_deen', 'wmt19_fien', 'wmt19_lten', 'wmt19_ruen',
    ],
    'XComet-XCOMET-XXL': [
        'wmt14_csen', 'wmt14_deen', 'wmt14_ruen', 'wmt14_fren',
        'wmt19_deen', 'wmt19_fien', 'wmt19_lten', 'wmt19_ruen',
    ],
    'Comet-wmt22-comet-da': [
        'wmt14_csen', 'wmt14_deen', 'wmt14_ruen', 'wmt14_fren',
        'wmt19_deen', 'wmt19_fien', 'wmt19_lten', 'wmt19_ruen',
    ],
}

METRICS = {
    'metricx-metricx-24-hybrid-xxl-v2p6': 'MetricX XXL',
    'XComet-XCOMET-XXL': 'XComet XXL',
    'Comet-wmt22-comet-da': 'Comet WMT22',
}

def main():
    rows = []  # columns: model, dataset, metric, method, prr_score, correction
    coef_rows = []  # columns: model, dataset, metric, method, degree, coef_idx, coef_val

    for metric_key, metric_name in METRICS.items():
        datasets = DATASETS[metric_key]
        ue_methods = list(methods_dict.values())

        for model_key, model_name in MODELS.items():
            ue_scores, ue_coefs, _ = detrend_ue_degreed(
                datasets,
                model_key,
                [metric_key],
                ue_methods,
                methods_dict
            )

            # Flatten scores
            for method_full, method_short in methods_dict.items():
                method_keys = [
                    (f"{method_short}_raw", "RAW"),
                    (f"{method_short}_deg1", "DEG1"),
                    (f"{method_short}_deg2", "DEG2"),
                    (f"{method_short}_deg3", "DEG3"),
                ]

                # Each key maps to a list aligned with `datasets`
                for key, corr in method_keys:
                    scores = ue_scores.get(key, [])
                    for dataset_name, score in zip(datasets, scores):
                        rows.append({
                            "model": model_name,
                            "dataset": dataset_name,
                            "metric": metric_name,
                            "method": method_short,
                            "correction": corr,
                            "prr_score": float(score),
                        })

                # Optionally also flatten the regression coefficients per degree
                for deg in (1, 2, 3):
                    coef_key = f"{method_short}_deg{deg}"
                    coef_list = ue_coefs.get(coef_key, [])
                    # coef_list has one coef array per dataset (matching order in `datasets`)
                    for dataset_name, coef_arr in zip(datasets, coef_list):
                        # Some degrees have 1 feature, some more (poly features)
                        for i, c in enumerate(np.ravel(coef_arr)):
                            coef_rows.append({
                                "model": model_name,
                                "dataset": dataset_name,
                                "metric": metric_name,
                                "method": method_short,
                                "degree": deg,
                                "coef_idx": i,
                                "coef_val": float(c),
                            })

    # Write the main experimental results
    df = pd.DataFrame(rows, columns=["model", "dataset", "metric", "method", "correction", "prr_score"])
    out_dir = Path("results")
    out_dir.mkdir(parents=True, exist_ok=True)
    out_path = out_dir / "mt_polynomials_results.csv"
    df.to_csv(out_path, index=False)
    print(f"Wrote {out_path.resolve()} with {len(df)} rows.")

if __name__ == "__main__":
    main()


In [ ]:
import pandas as pd

df = pd.read_csv("results/mt_polynomials_results.csv")


CORR_ORDER = ["RAW", "DEG1", "DEG2", "DEG3"]
CORR_ORDER_MAP = {c: i for i, c in enumerate(CORR_ORDER)}

def format_table_for_model_metric(df, model_name, metric_name):
    sub = df[(df["model"] == model_name) & (df["metric"] == metric_name)].copy()
    if sub.empty:
        return None

    sub["corr_order"] = sub["correction"].map(CORR_ORDER_MAP).fillna(999).astype(int)
    sub = sub.sort_values(["method", "corr_order"])
    sub["method_corr"] = sub["method"] + "-" + sub["correction"]

    pivot = sub.pivot_table(
        index="method_corr",
        columns="dataset",
        values="prr_score",
        aggfunc="mean"
    ).sort_index(key=lambda s: s.map(
        lambda x: (x.split("-", 1)[0], CORR_ORDER_MAP.get(x.split("-", 1)[1], 999))
    ))

    def style_top2(col: pd.Series):
        styles = [''] * len(col)
        valid = col.dropna()
        if valid.empty:
            return styles

        top_val = valid.max()
        second_candidates = valid[valid < top_val]
        second_val = second_candidates.max() if not second_candidates.empty else None

        for i, idx in enumerate(col.index):
            v = col.loc[idx]
            if pd.isna(v):
                continue
            if v == top_val:
                styles[i] = 'font-weight: bold;'
            elif second_val is not None and v == second_val:
                styles[i] = 'text-decoration: underline;'
        return styles

    styled = pivot.style.format("{:.2f}").apply(style_top2, axis=0)
    return styled

models = df["model"].dropna().unique()
metrics = df["metric"].dropna().unique()

for model in models:
    for metric in metrics:
        print(f"\n=== {model} | {metric} ===")
        st = format_table_for_model_metric(df, model, metric)
        if st is not None:
            display(st)
        else:
            print("(no data)")



=== Llama 3.1 8B | MetricX XXL ===


dataset,wmt14_csen,wmt14_deen,wmt14_fren,wmt14_ruen,wmt19_deen,wmt19_fien,wmt19_lten,wmt19_ruen
method_corr,,,,,,,,
LSRL-RAW,0.39,0.35,0.30,0.37,0.36,0.32,0.42,0.31
LSRL-DEG1,0.38,0.35,0.29,0.38,0.35,0.33,0.40,0.33
LSRL-DEG2,0.40,0.36,0.27,0.28,0.35,0.35,0.42,0.28
LSRL-DEG3,0.40,0.34,0.31,0.38,0.35,0.34,0.42,0.32
MCNSE-RAW,0.42,0.38,0.32,0.40,0.38,0.39,0.44,0.32
MCNSE-DEG1,0.42,0.38,0.34,0.43,0.39,0.39,0.44,0.36
MCNSE-DEG2,0.38,0.39,0.28,0.35,0.38,0.39,0.43,0.31
MCNSE-DEG3,0.45,0.39,0.34,0.43,0.39,0.40,0.46,0.35
MCSE-RAW,0.16,0.14,0.16,0.20,0.14,-0.00,0.07,0.21



=== Llama 3.1 8B | XComet XXL ===


dataset,wmt14_csen,wmt14_deen,wmt14_fren,wmt14_ruen,wmt19_deen,wmt19_fien,wmt19_lten,wmt19_ruen
method_corr,,,,,,,,
LSRL-RAW,0.34,0.35,0.27,0.29,0.30,0.30,0.32,0.30
LSRL-DEG1,0.33,0.31,0.24,0.38,0.25,0.33,0.34,0.33
LSRL-DEG2,0.35,0.35,0.21,0.20,0.27,0.35,0.35,0.25
LSRL-DEG3,0.35,0.27,0.27,0.32,0.28,0.34,0.35,0.32
MCNSE-RAW,0.36,0.34,0.27,0.32,0.33,0.38,0.43,0.32
MCNSE-DEG1,0.36,0.35,0.31,0.39,0.35,0.38,0.42,0.40
MCNSE-DEG2,0.31,0.39,0.23,0.27,0.31,0.38,0.41,0.31
MCNSE-DEG3,0.39,0.39,0.32,0.36,0.36,0.38,0.42,0.38
MCSE-RAW,0.20,0.29,0.29,0.33,0.24,-0.04,0.06,0.31



=== Llama 3.1 8B | Comet WMT22 ===


...

dataset,wmt14_csen,wmt14_deen,wmt14_fren,wmt14_ruen,wmt19_deen,wmt19_fien,wmt19_lten,wmt19_ruen
method_corr,,,,,,,,
LSRL-RAW,0.29,0.34,0.27,0.30,0.31,0.30,0.31,0.20
LSRL-DEG1,0.29,0.34,0.27,0.30,0.30,0.32,0.31,0.21
LSRL-DEG2,0.26,0.34,0.27,0.27,0.30,0.31,0.28,0.20
LSRL-DEG3,0.27,0.33,0.27,0.29,0.30,0.31,0.31,0.22
MCNSE-RAW,0.23,0.33,0.22,0.29,0.28,0.32,0.26,0.21
MCNSE-DEG1,0.21,0.33,0.21,0.28,0.26,0.30,0.25,0.23
MCNSE-DEG2,0.19,0.32,0.20,0.26,0.24,0.28,0.21,0.22
MCNSE-DEG3,0.19,0.32,0.19,0.29,0.23,0.28,0.24,0.24
MCSE-RAW,0.20,0.24,0.16,0.28,0.25,0.10,0.25,0.27



=== EuroLLM 9B | XComet XXL ===


dataset,wmt14_csen,wmt14_deen,wmt14_fren,wmt14_ruen,wmt19_deen,wmt19_fien,wmt19_lten,wmt19_ruen
method_corr,,,,,,,,
LSRL-RAW,0.23,0.28,0.23,0.22,0.28,0.24,0.18,0.18
LSRL-DEG1,0.23,0.28,0.23,0.26,0.28,0.26,0.18,0.22
LSRL-DEG2,0.19,0.27,0.23,0.17,0.28,0.25,0.16,0.19
LSRL-DEG3,0.20,0.27,0.23,0.22,0.27,0.25,0.19,0.22
MCNSE-RAW,0.19,0.28,0.17,0.23,0.26,0.30,0.22,0.21
MCNSE-DEG1,0.18,0.28,0.17,0.29,0.25,0.28,0.19,0.26
MCNSE-DEG2,0.15,0.27,0.16,0.19,0.22,0.25,0.14,0.21
MCNSE-DEG3,0.15,0.27,0.15,0.25,0.22,0.23,0.18,0.24
MCSE-RAW,0.19,0.31,0.26,0.35,0.30,-0.01,0.08,0.34



=== EuroLLM 9B | Comet WMT22 ===


dataset,wmt14_csen,wmt14_deen,wmt14_fren,wmt14_ruen,wmt19_deen,wmt19_fien,wmt19_lten,wmt19_ruen
method_corr,,,,,,,,
LSRL-RAW,0.32,0.38,0.29,0.31,0.40,0.35,0.32,0.26
LSRL-DEG1,0.32,0.37,0.29,0.36,0.41,0.35,0.31,0.29
LSRL-DEG2,0.29,0.37,0.29,0.31,0.40,0.35,0.30,0.28
LSRL-DEG3,0.30,0.37,0.29,0.32,0.40,0.35,0.31,0.30
MCNSE-RAW,0.23,0.36,0.22,0.28,0.34,0.36,0.28,0.24
MCNSE-DEG1,0.22,0.36,0.22,0.33,0.34,0.35,0.28,0.29
MCNSE-DEG2,0.20,0.35,0.21,0.28,0.31,0.33,0.25,0.28
MCNSE-DEG3,0.20,0.35,0.19,0.31,0.31,0.32,0.27,0.29
MCSE-RAW,0.35,0.36,0.28,0.42,0.41,0.21,0.34,0.42
